In [2]:
import os
from scipy import sparse
import pandas as pd
import numpy as np
import neural_network as nn
import LSTM
import datapreprocessing as dp
import torch
import torch.nn as nn
import torch.utils.data as data
import matplotlib.pyplot as plt

## Loading the data

In [15]:
def get_interactions():
    df_user_item_rating = pd.read_csv('cleaned_user_item_rating.csv')
    df_user_item_rating = df_user_item_rating.drop(columns=['Unnamed: 0'])[:10000]
    df_user_item_rating['reviewerID'] = pd.factorize(df_user_item_rating['reviewerID'])[0] + 1
    df_user_item_rating['asin'] = pd.factorize(df_user_item_rating['asin'])[0] + 1
    
    # number of unique users
    n_users = df_user_item_rating.reviewerID.unique().shape[0]
    #print('{} unique users'.format(n_users))
    
    # number of unique items
    n_items = df_user_item_rating.asin.unique().shape[0]
    #print('{} unique items'.format(n_items))
    
    interactions = np.zeros((n_users, n_items))
    for row in df_user_item_rating.itertuples():
        interactions[row[1] - 1, row[2] - 1] = row[3]
    return interactions, n_items

In [16]:
interactions, n_items = get_interactions()

Given the underwhelming performance of our matrix factorization model, we try a simple feedforward recommendation system instead. 


Input of this neural network is a pair of user and item represented by their IDs. Both user and item IDs first pass through an embedding layer. The output of the embedding layer, which are two embedding vectors, are then concatenated into one and passed into a linear network. 


Output of the linear network is one dimensional - representing the rating for the user-item pair. 

The model is fit the same way as the matrix factorization model and uses the standard PyTorch approach of forward passing, computing the loss, backpropagating and updating weights.

In [17]:
def train_test_split(interactions, n=2):
    """
    Split an interactions matrix into training and test sets.
    Parameters
    ----------
    interactions : np.ndarray
    n : int (default=2)
        Number of items to select / row to place into test.
    Returns
    -------
    train : np.ndarray
    test : np.ndarray
    """
    test = np.zeros(interactions.shape)
    train = interactions.copy()
    for user in range(interactions.shape[0]):
        if interactions[user, :].nonzero()[0].shape[0] > n:
            test_interactions = np.random.choice(interactions[user, :].nonzero()[0],
                                                 size=n,
                                                 replace=False)
            train[user, test_interactions] = 0.
            test[user, test_interactions] = interactions[user, test_interactions]

    # Test and training are truly disjoint
    assert(np.all((train * test) == 0))
    return train, test

In [18]:
def interactions_train_test_split():
    interactions, _ = get_interactions()
    train, test = train_test_split(interactions)
    train = sparse.coo_matrix(train)
    test = sparse.coo_matrix(test)
    return train, test

In [19]:
training_data, test_data = interactions_train_test_split()

In [20]:
# normalize into [0,1]
def normalize(ratings):
    ratings = ratings.copy()
    max_ratings = ratings.max()
    ratings = ratings*1.0/max_ratings
    return ratings

In [21]:
training_data.data = normalize(training_data.data)

In [22]:
def seq_rating(training_data):
    n_users = training_data.shape[0]
    sequence = []
    ratings = []
    for users in range(0, n_users):
        sequence.append(np.nonzero(training_data.toarray()[users])[0])
        ratings.append(training_data.toarray()[users][[np.nonzero(training_data.toarray()[users])[0]]])
    return sequence, ratings

In [23]:
sequence_list, ratings_list = seq_rating(training_data)

In [33]:
embedding_dim = 64
hidden_dim = 128
n_output = 1

# add one to represent padding when there is not enough history
model = LSTM.LSTMRating(embedding_dim, hidden_dim, n_items + 1, n_output)
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

#if os.path.isfile('LSTM.pt'):
#    model = torch.load('LSTM.pt')

def train(epoch, sequence_list, ratings_list):
    loss_hist = []
    loss_average_list = []
    t = 0
    for sequence, target_ratings in zip(sequence_list, ratings_list):
        t = t + 1
        model.zero_grad()
        # initialize hidden layers
        model.hidden = model.init_hidden()
        # convert sequence to PyTorch variables
        sequence_var = torch.LongTensor(sequence.astype('int64'))
        # forward pass
        ratings_scores = model(sequence_var).flatten()
        target_ratings_var = torch.FloatTensor(target_ratings.astype('float32'))
        # compute loss
        loss = loss_fn(ratings_scores, target_ratings_var)
        # backpropagate
        loss.backward()
        # update weights
        optimizer.step()
        # print
        if t % 7000 == 0:
            #running_loss = float(running_loss) + float(loss.item()[0])
            loss_hist.append(loss.item())
            #print(t, loss.item())
            loss_average = np.sum(loss_hist)/len(loss_hist)
            loss_average_list.append(loss_average)
            #print(loss_average)
            print('Train Epoch: {0}, {1}'.format(epoch, loss_average))
            
for epoch in range(1,10):
    train(epoch, sequence_list, ratings_list)
    # NN speichern
    torch.save(model,'LSTM.pt')
    
print('Finished')

Train Epoch: 1, 0.027963126078248024
Train Epoch: 2, 0.02094120904803276
Train Epoch: 3, 0.019772373139858246
Train Epoch: 4, 0.019391752779483795
Train Epoch: 5, 0.01922580599784851
Train Epoch: 6, 0.01913793757557869
Train Epoch: 7, 0.01908610761165619
Train Epoch: 8, 0.019056079909205437
Train Epoch: 9, 0.01902804896235466
Finished


In [ ]:
H1 = 128
D_out = 1
model = nn.DenseNet(n_users, n_items, n_factors=40,H1=H1,D_out=D_out)

In [ ]:
#ratings = df_user_item_rating.pivot(index='reviewerID', columns='asin', values='overall').fillna(0)

In [ ]:
def train_neural_network(model, ratings, lr=1e-6):
    loss_fn = torch.nn.MSELoss() 
    optimizer = torch.optim.SGD(model.parameters(),
                            lr=lr)

    # Sort our data
    ratings = np.array(ratings)
    users, items = ratings.nonzero()
    t = 0.0
    total_loss = torch.Tensor([0])
    p = np.random.permutation(len(users))
    users, items = users[p], items[p]
    loss_hist = []
    for user, item in zip(*(users, items)):
        t = t + 1
        # get user, item and rating data
        rating = torch.FloatTensor([ratings[user, item]])
        user = torch.LongTen
        sor([int(user)])
        item = torch.LongTensor([int(item)])
    
        # predict and calculate loss
        prediction = model.forward(user, item).flatten()
        loss = torch.sqrt(loss_fn(prediction, rating))
    
        # backpropagate
        loss.backward()

        # update weights
        optimizer.step()
    
        # Print
        if t % 100 == 0:
            #running_loss = float(running_loss) + float(loss.item()[0])
            loss_hist.append(loss.item())
            #print(t, loss.item())
            print(np.sum(loss_hist)/len(loss_hist))

In [ ]:
import torch.utils.data as data

class Interactions(data.Dataset):
    """
    Hold data in the form of an interactions matrix.
    Typical use-case is like a ratings matrix:
    - Users are the rows
    - Items are the columns
    - Elements of the matrix are the ratings given by a user for an item.
    """

    def __init__(self, mat):
        self.mat = mat.astype(np.float32).tocoo()
        self.n_users = self.mat.shape[0]
        self.n_items = self.mat.shape[1]

    def __getitem__(self, index):
        row = self.mat.row[index]
        col = self.mat.col[index]
        val = self.mat.data[index]
        return (row, col), val

    def __len__(self):
        return self.mat.nnz

In [ ]:
ratings = np.array(ratings)
for user in range(len(ratings)):
        user_name = np.ones(ratings.shape[1])*user
        user_name = user_name.astype(int)
        j = ratings[user,:]
        items = pd.DataFrame(ratings).columns.values
        A = sparse.csr_matrix(((user_name),(items,j)))

In [ ]:
#ratings[:100000].to_csv('ratings.csv')

In [ ]:
# My Hyperparameters
embedding_dim = 64
hidden_dim = 128
n_output = 1

# add one to represent padding when there is not enough history
model = LSTM.LSTMRating(embedding_dim, hidden_dim, n_items+1, n_output)
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

#if os.path.isfile('LSTM.pt'):
#    model = torch.load('LSTM.pt')

# Trainingsprozess
def train(epoch, ratings):
    model.train()
    loss_hist = []
    loss_average_list = []
    t = 0
    ratings = ratings.astype(int)
    ratings = np.array(ratings)
    for user in range(len(ratings)):
        t = t + 1
        user_name = np.ones(ratings.shape[1])*user
        user_name = user_name.astype(int)
        j = ratings[user,:]
        items = pd.DataFrame(ratings).columns.values
        A = sparse.csr_matrix(((user_name),(items,j)))
        print(A.tocoo().col)
        X_col = A.tocoo().col
        X_row = A.tocoo().row
        indices = np.nonzero(X_col)
        columns_non_unique = indices[0]
        unique_columns = sorted(set(columns_non_unique))
        user_item = A.tocoo().row[unique_columns]
        user_rating = A.tocoo().col[unique_columns]
        user = np.ones(len(user_item))*5
        user = user.astype(int)
        user_item_rating = sparse.csr_matrix(((user),(user_item,user_rating)))
        #print(user_item_rating)
        user_item_rating.tocoo().row
        
        sequence = user_item_rating.tocoo().row
        target_ratings = user_item_rating.tocoo().col
        #print(sequence)
        #print(target_ratings)
        # Set gardient zero
        model.zero_grad()
        # initialize hidden layers
        model.hidden = model.init_hidden()
        # convert sequence to PyTorch variables
        sequence_var = torch.LongTensor(sequence.astype('int64'))
        #print(sequence_var)
        # forward pass
        ratings_scores = model(sequence_var).flatten()
        target_ratings_var = torch.FloatTensor(target_ratings.astype('float32')).flatten()
        # compute loss
        loss = torch.sqrt(loss_fn(ratings_scores, target_ratings_var))
        # backpropagate
        loss.backward()
        # update weights
        optimizer.step()
        # print
        if t % 100 == 0:
            #running_loss = float(running_loss) + float(loss.item()[0])
            loss_hist.append(loss.item())
            #print(t, loss.item())
            loss_average = np.sum(loss_hist)/len(loss_hist)
            loss_average_list.append(loss_average)
            #print(loss_average)
            print('Train Epoch: {0}, {1}'.format(epoch, loss_average))
        
for epoch in range(1,3):
    train(epoch, interactions)
        
# NN speichern
torch.save(model,'LSTM.pt')

In [ ]:
def test():
    loss = 0

In [ ]:
plt.plot(loss_average_list)
plt.show()

In [ ]:
def bpr_loss(preds, vals):
    sig = nn.Sigmoid()
    return (1.0 - sig(preds)).pow(2).sum()